<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 2.2: Combinational Logic
**Prev: [Your First Chisel Module](2.1_first_module.ipynb)**<br>
**Next: [Control Flow](2.3_control_flow.ipynb)**

## Motivation
이 섹션에서는 Chisel 구성 요소를 사용하여 조합 논리를 구현하는 방법을 보여줍니다.
기본 Chisel 유형 중 세 가지 방법을 보여줍니다. `UInt `- 부호 없는 정수; `SInt` - 부호 있는 정수 및 `Bool` - true 또는 false가 연결되고 작동될 수 있습니다.
모든 Chisel 변수가 Scala `val`로 선언되는 방식에 주목하십시오.
한 번 정의되면 구성 자체가 변경되지 않을 수 있으므로 하드웨어 구성에 대해 Scala `var`를 사용하지 마십시오. 하드웨어를 실행할 때 값만 변경될 수 있습니다.
와이어는 매개변수화된 유형에 사용할 수 있습니다.

## Setup

In [1]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: String = "/home/parkdongho/dev/chisel-bootcamp/source/load-ivy.sc"

In [2]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test

import chisel3._

import chisel3.util._

import chisel3.tester._

import chisel3.tester.RawTester.test

---
# Common Operators
`Module`이 어떻게 구성되는지 이해했으므로 이제 하드웨어를 만들어 봅시다! 아래의 빈 모듈을 살펴보십시오.

In [3]:
class MyModule extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })
}

defined class MyModule

우리는 클래스를 `MyModule`이라고 불렀고 `Module`을 확장합니다. 이것은 Verilog의 하드웨어 모듈에 매핑됨을 의미합니다. 우리의 `MyModule` 모듈에는 하나의 입력과 하나의 출력이 있습니다. 입력은 4비트 부호 없는 정수(`UInt`)이고 출력도 마찬가지입니다.

<span style="color:blue">**Example: Scala and Chisel Operators Look the Same**</span><br>
데이터에 대해 수행할 수 있는 다양한 작업을 살펴보겠습니다.

In [8]:
class MyModule extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })

  val two  = 1 + 1
  println(two)
  val utwo = 1.U + 1.U
  println(utwo)
  
  io.out := io.in
}
println(getVerilog(new MyModule))

Elaborating design...
2
UInt<1>(OpResult in MyModule)
Done elaborating.
module MyModule(
  input        clock,
  input        reset,
  input  [3:0] io_in,
  output [3:0] io_out
);
  assign io_out = io_in; // @[cmd7.sc 12:10]
endmodule



defined class MyModule

우리는 두 개의 `val`을 생성합니다. 첫 번째는 두 개의 Scala `Int`를 추가하므로 `println`은 정수 2를 출력합니다. 두 번째 `val`은 두 개의 *Chisel* `UInt`를 함께 추가하므로 `println`은 이것을 하드웨어 노드로 보고 다음을 출력합니다. 유형 이름과 포인터(`chisel3.core.UInt@d`). `1.U`는 Scala `Int`(1)에서 Chisel `UInt` 리터럴로의 타입 캐스트입니다.

출력을 무언가로 drive해야 하므로 이전 자습서의 `passthrough` 모듈과 마찬가지로 지금은 입력에 연결하기만 하면 됩니다.

<span style="color:blue">**Example: Incompatible Operation**</span><br>
리터럴 `1`에 Chisel `1.U`를 추가하면 어떻게 됩니까? 전자는 값 1의 하드웨어 와이어이고 후자는 Scala 값 1이므로 이러한 타입은 호환되지 않습니다. 따라서 Chisel은 타입 불일치 오류를 제공합니다.

In [3]:
class MyModuleTwo extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })

  val twotwo = 1.U + 1
  println(twotwo)
  
  io.out := io.in
}
println(getVerilog(new MyModuleTwo))

cmd3.sc:7: type mismatch;
 found   : Int(1)
 required: chisel3.UInt
  val twotwo = 1.U + 1
                     ^Compilation Failed

: 

작업을 수행할 때 타입 간의 차이점을 기억하는 것이 중요합니다. Scala는 강력한 타입 언어이므로 모든 타입 캐스팅은 명시적이어야 합니다.

<span style="color:blue">**Example: More Chisel Operators**</span><br>
다른 일반적인 연산은 빼기와 곱하기입니다. 이들은 예상대로 부호 없는 정수에서 처리됩니다. 이러한 작업을 살펴보겠습니다. Verilog를 보여주지만 우리가 예상하는 간단한 코드를 난독화하는 몇 가지 기본 Chisel 기능이 있습니다.

In [9]:
class MyOperators extends Module {
  val io = IO(new Bundle {
    val in      = Input(UInt(4.W))
    val out_add = Output(UInt(4.W))
    val out_sub = Output(UInt(4.W))
    val out_mul = Output(UInt(4.W))
  })

  io.out_add := 1.U + 4.U
  io.out_sub := 2.U - 1.U
  io.out_mul := 4.U * 2.U
}
println(getVerilog(new MyOperators))

Elaborating design...
Done elaborating.
module MyOperators(
  input        clock,
  input        reset,
  input  [3:0] io_in,
  output [3:0] io_out_add,
  output [3:0] io_out_sub,
  output [3:0] io_out_mul
);
  wire [1:0] _T_3 = 2'h2 - 2'h1; // @[cmd8.sc 10:21]
  wire [4:0] _T_4 = 3'h4 * 2'h2; // @[cmd8.sc 11:21]
  assign io_out_add = 4'h5; // @[cmd8.sc 9:21]
  assign io_out_sub = {{2'd0}, _T_3}; // @[cmd8.sc 10:21]
  assign io_out_mul = _T_4[3:0]; // @[cmd8.sc 11:14]
endmodule



defined class MyOperators

And here's a sample tester for the above operations. Instead of using an anonymous tester class like in the previous tutorial, we'll create an explicit tester class. This is just an alternative way of writing a tester.

In [10]:
test(new MyOperators) {c =>
  c.io.out_add.expect(5.U)
  c.io.out_sub.expect(1.U)
  c.io.out_mul.expect(8.U)
}
println("SUCCESS!!")

Elaborating design...
Done elaborating.
test MyOperators Success: 0 tests passed in 2 cycles in 0.001250 seconds 1599.82 Hz
SUCCESS!!


<span style="color:blue">**Example: Mux and Concatenation**</span><br>
In addition to addition, subtraction, and multplication, Chisel has mux and concatenation operators. These are shown below. The `Mux` operates like a traditional ternary operator, with the order (select, value if true, value if false). Note that `true.B` and `false.B` are the preferred ways to create Chisel Bool literals. The `Cat` ordering is MSB then LSB (where B refers to bit or bits), and only takes two arguments. Concatenating more than two values requires multiple `Cat` calls or advanced Chisel and Scala features covered in later sections.

In [11]:
class MyOperatorsTwo extends Module {
  val io = IO(new Bundle {
    val in      = Input(UInt(4.W))
    val out_mux = Output(UInt(4.W))
    val out_cat = Output(UInt(4.W))
  })

  val s = true.B
  io.out_mux := Mux(s, 3.U, 0.U) // should return 3.U, since s is true
  io.out_cat := Cat(2.U, 1.U)    // concatenates 2 (b10) with 1 (b1) to give 5 (101)
}

println(getVerilog(new MyOperatorsTwo))

test(new MyOperatorsTwo) { c =>
  c.io.out_mux.expect(3.U)
  c.io.out_cat.expect(5.U)
}
println("SUCCESS!!")

Elaborating design...
Done elaborating.
module MyOperatorsTwo(
  input        clock,
  input        reset,
  input  [3:0] io_in,
  output [3:0] io_out_mux,
  output [3:0] io_out_cat
);
  assign io_out_mux = 4'h3; // @[cmd10.sc 9:20]
  assign io_out_cat = 4'h5; // @[Cat.scala 30:58]
endmodule

Elaborating design...
Done elaborating.
test MyOperatorsTwo Success: 0 tests passed in 2 cycles in 0.001010 seconds 1979.85 Hz
SUCCESS!!


defined class MyOperatorsTwo

Notice how the Verilog contains constants instead of actual mux or concatenation logic. This is because FIRRTL transformations have simplified the circuit, eliminating obvious logic. 

For a more complete list of Chisel operators, see the [Chisel cheatsheet](https://github.com/freechipsproject/chisel-cheatsheet/releases/latest/download/chisel_cheatsheet.pdf). For the most complete list of operators and their implementation details, look through the [Chisel API](https://chisel-lang.org/api/latest/).

---
# Exercises
To complete these exercises, you may need to look through the [Chisel cheatsheet](https://github.com/freechipsproject/chisel-cheatsheet/releases/latest/download/chisel_cheatsheet.pdf).

<span style="color:red">**Exercise: MAC**</span><br>
Create a Chisel module that implements the multiply accumulate function, `(A*B)+C`, and passes the testbench.

In [4]:
class MAC extends Module {
  val io = IO(new Bundle {
    val in_a = Input(UInt(4.W))
    val in_b = Input(UInt(4.W))
    val in_c = Input(UInt(4.W))
    val out  = Output(UInt(8.W))
  })

  ???
}

test(new MAC) { c =>
  val cycles = 100
  import scala.util.Random
  for (i <- 0 until cycles) {
    val in_a = Random.nextInt(16)
    val in_b = Random.nextInt(16)
    val in_c = Random.nextInt(16)
    c.io.in_a.poke(in_a.U)
    c.io.in_b.poke(in_b.U)
    c.io.in_c.poke(in_c.U)
    c.io.out.expect((in_a * in_b + in_c).U)
  }
}
println("SUCCESS!!")

Elaborating design...
Done elaborating.
test MAC Success: 0 tests passed in 2 cycles in 0.037067 seconds 53.96 Hz
SUCCESS!!


defined class MAC

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
class MAC extends Module {
  val io = IO(new Bundle {
    val in_a = Input(UInt(4.W))
    val in_b = Input(UInt(4.W))
    val in_c = Input(UInt(4.W))
    val out  = Output(UInt(8.W))
  })

  io.out := (io.in_a * io.in_b) + io.in_c
}
</pre></article></div></section></div>

<span style="color:red">**Exercise: Arbiter**</span><br>
The following circuit arbitrates data coming from a FIFO into two parallel processing units. The FIFO and processing elements (PEs) communicate with ready-valid interfaces. Construct the arbiter to send data to whichever PE is ready to receive data, prioritizing PE0 if both are ready to receive data. Remember that the arbiter should tell the FIFO that it's ready to receive data when at least one of the PEs can receive data. Also, wait for a PE to assert that it's ready before asserting that the data are valid. You will likely need binary operators to complete this exercise.  
다음 회로는 FIFO에서 오는 데이터를 두 개의 병렬 처리 장치로 중재합니다. FIFO 및 처리 요소(PE)는 즉시 사용 가능한 인터페이스와 통신합니다. 데이터를 수신할 준비가 된 PE에 데이터를 보내도록 중재자를 구성하고 둘 다 데이터를 수신할 준비가 된 경우 PE0에 우선순위를 둡니다. 중재자는 PE 중 적어도 하나가 데이터를 수신할 수 있을 때 데이터를 수신할 준비가 되었음을 FIFO에 알려야 합니다. 또한 데이터가 유효하다고 주장하기 전에 PE가 준비가 되었다고 주장할 때까지 기다리십시오. 이 연습을 완료하려면 이항 연산자가 필요할 수 있습니다.

<img src="images/arbiter.png" width="687" height="177">

In [4]:
class Arbiter extends Module {
  val io = IO(new Bundle {
    // FIFO
    val fifo_valid = Input(Bool())
    val fifo_ready = Output(Bool())
    val fifo_data  = Input(UInt(16.W))
    
    // PE0
    val pe0_valid  = Output(Bool())
    val pe0_ready  = Input(Bool())
    val pe0_data   = Output(UInt(16.W))
    
    // PE1
    val pe1_valid  = Output(Bool())
    val pe1_ready  = Input(Bool())
    val pe1_data   = Output(UInt(16.W))
  })
    
  ???
    
}

test(new Arbiter) { c =>
  import scala.util.Random
  val data = Random.nextInt(65536)
  c.io.fifo_data.poke(data.U)
  
  for (i <- 0 until 8) {
    c.io.fifo_valid.poke((((i >> 0) % 2) != 0).B)
    c.io.pe0_ready.poke((((i >> 1) % 2) != 0).B)
    c.io.pe1_ready.poke((((i >> 2) % 2) != 0).B)

    c.io.fifo_ready.expect((i > 1).B)
    c.io.pe0_valid.expect((i == 3 || i == 7).B)
    c.io.pe1_valid.expect((i == 5).B)
    
    if (i == 3 || i ==7) {
      c.io.pe0_data.expect((data).U)
    } else if (i == 5) {
      c.io.pe1_data.expect((data).U)
    }
  }
}
println("SUCCESS!!")

Elaborating design...
Done elaborating.
test Arbiter Success: 0 tests passed in 2 cycles in 0.022534 seconds 88.76 Hz
SUCCESS!!


defined class Arbiter

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-2" />
<label for="check-2"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  io.fifo_ready := io.pe0_ready || io.pe1_ready
  io.pe0_valid := io.fifo_valid && io.pe0_ready
  io.pe1_valid := io.fifo_valid && io.pe1_ready && !io.pe0_ready
  io.pe0_data := io.fifo_data
  io.pe1_data := io.fifo_data
</pre></article></div></section></div>

<span style="color:red">**Exercise: Parameterized Adder (Optional)**</span><br>

이 선택적인 연습은 Chisel의 가장 강력한 기능 중 하나인 매개변수화 기능을 보여줍니다. 이것을 보여주기 위해 우리는 오버플로가 발생할 때 출력을 포화시키거나 결과를 잘라낼 수 있는 매개변수화된 가산기를 구성할 것입니다(즉, wrap around).

먼저 아래의 `Module`을 보자. 우리가 전달하는 매개변수는 `saturate`라고 하며 유형은 *Scala* `Boolean`입니다. 이것은 Chisel `Bool`이 아닙니다. 따라서 우리는 포화 또는 절단할 수 있는 단일 하드웨어 가산기를 만드는 것이 아니라 포화 하드웨어 가산기 *또는* 절단 하드웨어 가산기를 생성하는 *generator*를 만들고 있습니다. 결정은 컴파일 시간에 이루어집니다.

다음으로, 입력과 출력이 모두 4비트 `UInt`입니다. Chisel에는 너비 추론 기능이 내장되어 있으며 [cheatsheet](https://github.com/freechipsproject/chisel-cheatsheet/releases/latest/download/chisel_cheatsheet.pdf)를 보면 비트폭이 일반 합산의 최대 비트폭은 두 입력의 최대 비트폭과 같습니다. 이것은 다음을 의미합니다

```scala
val sum = io.in_a + io.in_b
```

`sum`을 4비트 와이어로 만들고 값은 4비트 입력에 대해 잘린 결과가 됩니다. 합이 포화되어야 하는지 확인하려면 결과를 5비트 와이어에 배치해야 합니다. 이것은 치트시트에서 볼 수 있듯이 `+&` 합계로 수행할 수 있습니다.

```scala
val sum = io.in_a +& io.in_b
```

마지막으로 4비트 `UInt` 와이어를 5비트 `UInt` 와이어에 연결하면 기본적으로 MSB가 잘립니다. 이것을 사용하여 비포화 가산기에 대한 5비트 합을 쉽게 자를 수 있습니다.

In [6]:
class ParameterizedAdder(saturate: Boolean) extends Module {
  val io = IO(new Bundle {
    val in_a = Input(UInt(4.W))
    val in_b = Input(UInt(4.W))
    val out  = Output(UInt(4.W))
  })

  val sum = io.in_a +& io.in_b
  if (saturate) {
    io.out := Mux(sum > 15.U, 15.U, sum)
  } else {
    io.out := sum
  }
}

for (saturate <- Seq(true, false)) {
  test(new ParameterizedAdder(saturate)) { c =>
    // 100 random tests
    val cycles = 100
    import scala.util.Random
    import scala.math.min
    for (i <- 0 until cycles) {
      val in_a = Random.nextInt(16)
      val in_b = Random.nextInt(16)
      c.io.in_a.poke(in_a.U)
      c.io.in_b.poke(in_b.U)
      if (saturate) {
        c.io.out.expect(min(in_a + in_b, 15).U)
      } else {
        c.io.out.expect(((in_a + in_b) % 16).U)
      }
    }
    
    // ensure we test saturation vs. truncation
    c.io.in_a.poke(15.U)
    c.io.in_b.poke(15.U)
    if (saturate) {
      c.io.out.expect(15.U)
    } else {
      c.io.out.expect(14.U)
    }
  }
}
println("SUCCESS!!")

Elaborating design...
Done elaborating.
test ParameterizedAdder Success: 0 tests passed in 2 cycles in 0.013929 seconds 143.59 Hz
Elaborating design...
Done elaborating.
test ParameterizedAdder Success: 0 tests passed in 2 cycles in 0.006871 seconds 291.08 Hz
SUCCESS!!


defined class ParameterizedAdder

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-3" />
<label for="check-3"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  val sum = io.in_a +& io.in_b
  if (saturate) {
    io.out := Mux(sum > 15.U, 15.U, sum)
  } else {
    io.out := sum
  }
</pre></article></div></section></div>

---
# You're done!

[Return to the top.](#top)